In [1]:
import cv2
import mediapipe as mp
import numpy as np
from tensorflow.keras.models import load_model

In [2]:
actions = ['head', 'shoulder', 'knee', 'clap', 'left_foot', 'right_foot', 'both_foot', 'yeah']
seq_length = 30

In [3]:
model = load_model('./models/model.h5')

In [4]:
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(static_image_mode=False,
                    model_complexity=1,
                    smooth_landmarks=True,
                    enable_segmentation=False,
                    smooth_segmentation=True,
                    min_detection_confidence=0.5,
                    min_tracking_confidence=0.5)

# 영상으로 확인

In [25]:
#cap = cv2.VideoCapture(0)
cap = cv2.VideoCapture('./pinkpong33s.mp4')

seq = []
action_seq = []

while cap.isOpened():
    ret, img = cap.read()
    #img0 = img.copy()
    
    img = cv2.resize(img, (0,0), fx=1/2, fy=1/2, interpolation=cv2.INTER_AREA)
    img = cv2.flip(img, 1)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    result = pose.process(img)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

    if result.pose_landmarks is not None:
        joint = np.zeros((33, 4))
        for j, res in enumerate(result.pose_landmarks.landmark):
            joint[j] = [res.x, res.y, res.z, res.visibility]

        # Compute angles between joints
        v1 = joint[[13, 23, 14, 24, 15, 16, 17, 19, 21, 18, 20, 22, 25, 26, 27, 28, 29, 31, 30, 32], :3] # Parent joint
        v2 = joint[[11, 11, 12, 12, 13, 14, 15, 15, 15, 16, 16, 16, 23, 24, 25, 26, 27, 27, 28, 28], :3] # Child joint
        v = v2 - v1 # [20, 3]
        # Normalize v
        v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]
        v = np.nan_to_num(v)

        # Get angle using arcos of dot product
        angle = np.arccos(np.einsum('nt,nt->n',
            v[[0, 2, 0, 2, 4, 5, 7, 10, 1, 3, 12, 13, 14, 14, 15, 15],:], 
            v[[1, 3, 4, 5, 6, 9, 8, 11, 12, 13, 14, 15, 16, 17, 18, 19],:])) # [16,]

        angle = np.degrees(angle) # Convert radian to degree

        d = np.concatenate([joint.flatten(), angle])

        seq.append(d)

        mp_drawing.draw_landmarks(img, result.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        if len(seq) < seq_length:
            continue

        input_data = np.expand_dims(np.array(seq[-seq_length:], dtype=np.float32), axis=0)

        y_pred = model.predict(input_data).squeeze()

        i_pred = int(np.argmax(y_pred))
        conf = y_pred[i_pred]

        if conf < 0.9:
            continue

        action = actions[i_pred]
        action_seq.append(action)

        if len(action_seq) < 3:
            continue

        this_action = '?'
        if action_seq[-1]:
            this_action = action

        cv2.putText(img, f'{this_action.upper()}', org=(int(res.x * img.shape[1]), int(res.y * img.shape[0] + 20)), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(255, 255, 255), thickness=2)

    # out.write(img0)Q
    # out2.write(img)
    cv2.imshow('img', img)
    if cv2.waitKey(1) == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 43ms/step


1/1 [==============================] - 0s 41ms/step


1/1 [==============================] - 0s 26ms/step


In [ ]:
cap.release()
cv2.destroyAllWindows()

# skeleton 으로 확인

In [10]:
import pandas as pd

In [12]:
cam = pd.read_csv("./cam_skeleton.csv", header=[0,1])

In [14]:
seq = []
action_seq = []

cam_array = np.array(cam).reshape(382, 33, 4)

for joint in cam_array:
    img = np.full((720, 640, 3), 255, np.uint8)
    # Compute angles between joints
    v1 = joint[[13, 23, 14, 24, 15, 16, 17, 19, 21, 18, 20, 22, 25, 26, 27, 28, 29, 31, 30, 32], :3] # Parent joint
    v2 = joint[[11, 11, 12, 12, 13, 14, 15, 15, 15, 16, 16, 16, 23, 24, 25, 26, 27, 27, 28, 28], :3] # Child joint
    v = v2 - v1 # [20, 3]
    # Normalize v
    v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]
    v = np.nan_to_num(v)

    # Get angle using arcos of dot product
    angle = np.arccos(np.einsum('nt,nt->n',
        v[[0, 2, 0, 2, 4, 5, 7, 10, 1, 3, 12, 13, 14, 14, 15, 15],:], 
        v[[1, 3, 4, 5, 6, 9, 8, 11, 12, 13, 14, 15, 16, 17, 18, 19],:])) # [16,]

    angle = np.degrees(angle) # Convert radian to degree

    d = np.concatenate([joint.flatten(), angle])

    seq.append(d)
    
    for x, y in zip(joint[:,0], joint[:,1]):
        cv2.circle(img, (round(x*640), round(y*720)), 3, (0, 0, 0), -1)
    
    cv2.imshow('img', img)
    
    if len(seq) < seq_length:
        continue

    input_data = np.expand_dims(np.array(seq[-seq_length:], dtype=np.float32), axis=0)

    y_pred = model.predict(input_data).squeeze()

    i_pred = int(np.argmax(y_pred))
    conf = y_pred[i_pred]

    if conf < 0.9:
        continue

    action = actions[i_pred]
    action_seq.append(action)

    if len(action_seq) < 3:
        continue

    this_action = '?'
    if action_seq[-1] == action_seq[-2] == action_seq[-3]:
        this_action = action
    
    cv2.putText(img, f'{this_action.upper()}', org=(0, 720), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(0, 0, 0), thickness=2)

    # out.write(img0)
    # out2.write(img)
    cv2.imshow('img', img)
    if cv2.waitKey(33) == ord('q'):
        break
        
cv2.destroyAllWindows()

1/1 [==============================] - 0s 24ms/step
